#  Variation of the times with the network length #

In [1]:
%matplotlib inline

from LinearChain import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from scipy.integrate import ode
from scipy.integrate import quad
from scipy import stats

## Parameters for the runs ##

In [2]:
N_max = 100
conc_ref = 1
a = 2
b = 1
KS = 2
KP = 2
α = a*KS
β = b*KP
S = 1 + conc_ref/KS + conc_ref/KP
F = (α*conc_ref/KS-β*conc_ref/KP)/(1 + conc_ref/KS + conc_ref/KP)

In [3]:
MA_conc_data = []
MM_conc_data = []
MM_tracer_data = []
Ns = np.arange(2,N_max)
for n in range(2,N_max):
    conc_st = np.ones(n+2)*conc_ref
    param_vec_MA = np.array([a,b]*(n+1))
    param_vec_MM = np.array([α,β,KS,KP]*(n+1))
    MA_conc_data.append(run_homogenous_model(conc_st,param_vec_MA,typeofmodel="MA-conc",task_list=["relax_time","transit_time_L1"]))    
    MM_tracer_data.append(run_homogenous_model(conc_st,param_vec_MM,typeofmodel="MM-tracer",task_list=["relax_time","transit_time_L1"]))
    MM_conc_data.append(run_homogenous_model(conc_st,param_vec_MM,typeofmodel="MM-conc",task_list=["relax_time","transit_time_L1"]))    
MA_conc_data = np.array(MA_conc_data)
MM_tracer_data = np.array(MM_tracer_data)
MM_conc_data = np.array(MM_conc_data)

TypeError: only integer arrays with one element can be converted to an index